# Блок 1

In [18]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [6]:
!hdfs dfsadmin -safemode leave

Safe mode is OFF


In [7]:
conf = SparkConf().set("spark.executor.instances", "2").set("spark.executor.cores", "1").set("spark.executor.memory", "1g")

In [8]:
spark = SparkSession.builder.appName("lukyanov_spark").config(conf=conf).master(master="yarn").getOrCreate()

23/12/15 17:02:40 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServ

In [10]:
!hdfs dfs -rm -r ml-latest-small
!hdfs dfs -put ml-latest-small

rm: `ml-latest-small': No such file or directory


In [11]:
ratings_df = spark.read.format("csv").option("header", "True").load("ml-latest-small/ratings.csv")

In [12]:
ratings_df.show(10)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
+------+-------+------+---------+
only showing top 10 rows



In [13]:
tags_df = spark.read.format("csv").option("header", "True").load("ml-latest-small/tags.csv")

In [14]:
tags_df.show(10)

+------+-------+-----------------+----------+
|userId|movieId|              tag| timestamp|
+------+-------+-----------------+----------+
|     2|  60756|            funny|1445714994|
|     2|  60756|  Highly quotable|1445714996|
|     2|  60756|     will ferrell|1445714992|
|     2|  89774|     Boxing story|1445715207|
|     2|  89774|              MMA|1445715200|
|     2|  89774|        Tom Hardy|1445715205|
|     2| 106782|            drugs|1445715054|
|     2| 106782|Leonardo DiCaprio|1445715051|
|     2| 106782|  Martin Scorsese|1445715056|
|     7|  48516|     way too long|1169687325|
+------+-------+-----------------+----------+
only showing top 10 rows



In [15]:
print(f'ratings-count: {ratings_df.count()}')
print(f'tags-count: {tags_df.count()}')

ratings-count: 100836
tags-count: 3683


# Блок 2

In [16]:
unique_movies_count = ratings_df.select("movieId").distinct().count()
unique_users_count = ratings_df.select("userId").distinct().count()
print(f"уникальных фильмов в ratings: {unique_movies_count}")
print(f"уникальных пользователей в ratings: {unique_users_count}")

уникальных фильмов в ratings: 9724
уникальных пользователей в ratings: 610


In [17]:
ratings4_count = ratings_df.select("rating").filter("rating>=4.0").count()
print(f"оценок >=4 в ratings: {ratings4_count}")

оценок >=4 в ratings: 48580


In [19]:
movies_df = spark.read.format("csv").option("header", "True").load("ml-latest-small/movies.csv")
movies_df.show(10)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
+-------+--------------------+--------------------+
only showing top 10 rows



In [63]:
top100 = ratings_df\
    .groupBy(f.col("movieId"))\
    .agg(f.avg("rating").alias("movie_rating")) # считаем рейтинг как среднее по всем оценкам
top100_names = top100.join(movies_df, on = ['movieId']).orderBy(f.col("movie_rating").desc())
print("Топ-100 лучших фильмов")
top100_names.show(100)

Топ-100 лучших фильмов
+-------+------------+--------------------+--------------------+
|movieId|movie_rating|               title|              genres|
+-------+------------+--------------------+--------------------+
| 102084|         5.0|Justice League: D...|Action|Animation|...|
| 126921|         5.0|The Fox and the H...|Adventure|Animati...|
|   5723|         5.0|Continental Divid...|      Comedy|Romance|
|    467|         5.0|Live Nude Girls (...|              Comedy|
|  95149|         5.0|Superman/Batman: ...|Action|Animation|...|
| 140627|         5.0|Battle For Sevast...|   Drama|Romance|War|
|    633|         5.0|Denise Calls Up (...|              Comedy|
| 147330|         5.0|Sherlock Holmes a...|               Crime|
| 103602|         5.0|Craig Ferguson: I...|  Comedy|Documentary|
|  96608|         5.0|Runaway Brain (19...|Animation|Comedy|...|
| 118270|         5.0|  Hellbenders (2012)|Comedy|Horror|Thr...|
|   1349|         5.0|Vampire in Venice...|              Horror|
| 

In [30]:
rt_table = ratings_df.join(tags_df, on = ["userId", "movieId"], how="inner")
print("Среднее")
res = rt_table.select(f.avg(tags_df['timestamp']-ratings_df['timestamp']))
res.show()
print("Среднее (как большее минус меньшее)")
res = rt_table.select(f.avg(f.abs(tags_df['timestamp']-ratings_df['timestamp'])))
res.show()

Среднее
+----------------------------+
|avg((timestamp - timestamp))|
+----------------------------+
|        2.6243727372266974E7|
+----------------------------+

Среднее (как большее минус меньшее)
+---------------------------------+
|avg(abs((timestamp - timestamp)))|
+---------------------------------+
|             2.9203715568469506E7|
+---------------------------------+



In [32]:
res = ratings_df.groupBy("userId").agg(f.avg("rating").alias("user_average_rating"))
#res = res.join('users', on = ['userId'])
res.show()

+------+-------------------+
|userId|user_average_rating|
+------+-------------------+
|   296|  4.166666666666667|
|   467|  3.409090909090909|
|   125|  3.859722222222222|
|   451| 3.7941176470588234|
|     7| 3.2302631578947367|
|    51| 3.7757660167130918|
|   124|               3.99|
|   447|  3.871794871794872|
|   591| 3.2777777777777777|
|   307| 2.6656410256410257|
|   475|  4.409677419354839|
|   574| 3.9565217391304346|
|   169|   4.24907063197026|
|   205| 3.8703703703703702|
|   334|  3.418831168831169|
|   544|  4.363636363636363|
|   577| 3.5652173913043477|
|   581|              4.375|
|   272|  3.629032258064516|
|   442|              1.275|
+------+-------------------+
only showing top 20 rows



In [33]:
# среднее по средним
res.select(f.avg("user_average_rating")).show()

+------------------------+
|avg(user_average_rating)|
+------------------------+
|      3.6572223377474007|
+------------------------+



# Блок 3

In [44]:
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from pyspark.sql.types import *

In [45]:
tags = tags_df.toPandas()
ratings = ratings_df.toPandas()
ratings_tags = rt_table.toPandas()

In [46]:
ratings_tags.head()

,userId,movieId,rating,timestamp,tag,timestamp
0,2,60756,5.0,1445714980,will ferrell,1445714992
1,2,60756,5.0,1445714980,Highly quotable,1445714996
2,2,60756,5.0,1445714980,funny,1445714994
3,2,89774,5.0,1445715189,Tom Hardy,1445715205
4,2,89774,5.0,1445715189,MMA,1445715200


In [47]:
tfidfVectorizer = TfidfVectorizer()
vectorized_tags = tfidfVectorizer.fit_transform(ratings_tags["tag"])

In [48]:
model = SGDRegressor()
model.fit(X=vectorized_tags, y=ratings_tags["rating"])

SGDRegressor()

In [59]:
@f.udf(DoubleType())
def get_rating_prediction(tag): 
    return model.predict(tfidfVectorizer.transform([tag])).item()

res = rt_table.withColumn("rating_prediction", get_rating_prediction("tag"))

In [60]:
res.show(50)

+------+-------+------+----------+--------------------+----------+------------------+
|userId|movieId|rating| timestamp|                 tag| timestamp| rating_prediction|
+------+-------+------+----------+--------------------+----------+------------------+
|     2|  60756|   5.0|1445714980|        will ferrell|1445714992| 4.030629163996057|
|     2|  60756|   5.0|1445714980|     Highly quotable|1445714996|3.9549035156673655|
|     2|  60756|   5.0|1445714980|               funny|1445714994| 4.367005879032952|
|     2|  89774|   5.0|1445715189|           Tom Hardy|1445715205|3.8352641694269183|
|     2|  89774|   5.0|1445715189|                 MMA|1445715200| 3.421944591622135|
|     2|  89774|   5.0|1445715189|        Boxing story|1445715207|3.9309767156208695|
|     2| 106782|   5.0|1445714966|     Martin Scorsese|1445715056|3.8485335925683386|
|     2| 106782|   5.0|1445714966|   Leonardo DiCaprio|1445715051| 4.239562115250214|
|     2| 106782|   5.0|1445714966|               drugs